In [11]:
"""
Write sample initial inventory

"""


'\nWrite sample initial inventory\n\n'

In [12]:
# Import
import pandas as pd
import numpy as np

In [13]:
# Read starting inventory file


In [14]:
# Read transaction file
dT = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\transaction_data_2021_12_15.csv')
# Drop useless columns
dT = dT.iloc[:, 1:3+1]
# Rename columns
dT.columns = ['sourcePort', 'destPort', 'DMT']

dT.head()

,sourcePort,destPort,DMT
0,10,210,"122,78"
1,10,210,"19,04"
2,10,210,"235,14"
3,10010,10,"435,40"
4,10010,47,"31,10"


In [15]:
# Convert comma to decimal
dT.replace(',','.', regex=True, inplace=True)
# Convert to float
dT.iloc[:,-1] = pd.to_numeric(dT.iloc[:,-1])
dT.head()

,sourcePort,destPort,DMT
0,10,210,122.78
1,10,210,19.04
2,10,210,235.14
3,10010,10,435.40
4,10010,47,31.10


In [48]:
# Keep rows if destination is to "Eglise"
# Todo eglisePorts read from csv
eglisePorts = [i for i in range(900,950)]
eglisePorts.extend([564,565])
dT = dT[dT['destPort'].isin(eglisePorts)]
dT.head()

,sourcePort,destPort,DMT,shipperID
70,304,918,93.68,BCM
71,304,919,57.62,BCM
80,498,918,101.86,PYR
81,498,919,41.35,PYR
90,521,920,32.39,NMP


In [17]:
# Assign a shipper code to source port number
shipID = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\sourcePort_to_shipperId.csv')
shipID.head()


,ShipperID,SourcePort
0,BCM,304
1,PYR,498
2,NMP,520
3,NMP,521


In [18]:
# Create dictionary
shipID_dict = shipID.set_index('SourcePort')['ShipperID'].to_dict()
# This also works
# shipID_dict = dict(zip(shipID['SourcePort'], shipID['ShipperID']))

In [30]:
# Map shipperID to transactions
dT['shipperID'] = dT['sourcePort'].map(shipID_dict)
dT = dT[dT['shipperID'].notnull()]
dT.head()

C:\Users\win7\anaconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sourcePort,destPort,DMT,shipperID
70,304,918,93.68,BCM
71,304,919,57.62,BCM
80,498,918,101.86,PYR
81,498,919,41.35,PYR
90,521,920,32.39,NMP


In [22]:
# Read prior inventory
inv = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\inv.csv', index_col=0)
inv = inv.fillna(0)

inv

,564,565,900,903,906,913,915,916,917,918,919,920,924
XKM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FLI,0.0,0.0,682.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MTA,0.0,0.0,959.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0
Pyrite_don_rouyn_gallen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0
BCM,0.0,0.0,360.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DUM,0.0,0.0,0.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NVA,0.0,0.0,0.0,132.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
LSN_et_INC4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TIZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# Add transactions to inventory
# This is the naive method, find a more elegant way later.
for j in inv.columns:
    for i in inv.index:
        for k in range(dT.shape[0]):
            if dT.iloc[k,-1] == i and str(dT.iloc[k,1])==j:
                inv.loc[i,j] += float(dT.iloc[k,2])
                print('Added '+str(dT.iloc[k,2])+' tons of '+str(i)+' to port '+str(j)+'.')
                
inv.head()


Added 93.68 tons of BCM to port 918
Added 57.62 tons of BCM to port 919
Added 32.39 tons of NMP to port 920


,564,565,900,903,906,913,915,916,917,918,919,920,924
XKM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
FLI,0.0,0.0,682.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
MTA,0.0,0.0,959.0,0.0,0.0,0.0,0.0,0.0,22.0,0.00,0.00,0.0,0.0
Pyrite_don_rouyn_gallen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.00,0.00,0.0,0.0
BCM,0.0,0.0,360.0,0.0,0.0,0.0,0.0,0.0,0.0,93.68,57.62,0.0,0.0


In [ ]:
# Read smelted transactions
smelt = pd.read_csv()